In [ ]:
import sys
sys.path.append('..')

In [23]:
import glob, os, time
import json
import argparse

from keras.models import Model
from keras.optimizers import SGD, Adam
from keras import callbacks
from keras.initializers import VarianceScaling

from metadec.dataset.genome import SimGenomeDataset
from metadec.model.dec import DEC
from metadec.utils.utils import load_genomics
from metadec.debug.visualize import store_results
from metadec.utils.metrics import genome_acc

import tqdm

#### Hyperparameter

##### Training

In [ ]:
RESULT_DIR = '../results'
GEN_DATA_DIR = '../data'

# Versioning each runs
ARCH = 'dec_genomics'
DATE = '20200530'

# Training batchsize
BATCH_SIZE = 1
# Maximum iterations for clustering optimization step
MAX_ITERS = 10
# Number of epochs for pretraining
PRETRAIN_EPOCHS = 5
# Interval for updating the training status
UPDATE_INTERVAL = 1
# Tolerance threshold to stop clustering optimization step
TOL = 0.000001
# Trained weight for pretrained autoencoder
AE_WEIGHTS = None
# Dir contains raw fasta data
DATASET_DIR = GEN_DATA_DIR
# Specifc dataset or all of them
DATASET_NAME = 'S1'

##### Graph 

In [ ]:
# Hyperparameters
# Follows metaprob
KMERS = [4]
LMER = 30
NUM_SHARED_READS = (5, 45)
ONLY_SEED = True
MAXIMUM_SEED_SIZE = 9000

#### Initialize dirs

In [ ]:
LOG_DIR = f'{RESULT_DIR}\log'
MODEL_DIR = f'{RESULT_DIR}\model'
META_INFO =  '../config/dataset_metadata.json'


for d in [LOG_DIR, MODEL_DIR]:
    if not os.path.exists(d):
        os.makedirs(d)

# Dir for saving training results
SAVE_DIR = os.path.join(MODEL_DIR, ARCH, DATE)

# Dir for saving log results: visualization, training logs
LOG_DIR = os.path.join(LOG_DIR, ARCH, DATE)

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

#### Loading metadata

In [ ]:
raw_dir = os.path.join(DATASET_DIR, 'raw')
processed_dir = os.path.join(DATASET_DIR, 'processed')
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
if DATASET_NAME == 'all':
    raw_datasets = glob.glob(raw_dir + '/*.fna')
else:
    raw_datasets = [os.path.join(raw_dir, DATASET_NAME + '.fna')]

# Mapping of dataset and its corresponding number of clusters
with open(META_INFO, 'r') as f:
    n_clusters_mapping = json.load(f)['simulated']

In [ ]:
raw_datasets

#### Running

In [31]:
for dataset in tqdm.tqdm(raw_datasets):
    # Get some parameters
    dataset_name = os.path.basename(dataset).split('.fna')[0]
    save_dir = os.path.join(SAVE_DIR, dataset_name)
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    num_shared_read = NUM_SHARED_READS[1] if 'R' in dataset_name else NUM_SHARED_READS[0]
    is_deserialize = os.path.exists(os.path.join(processed_dir, dataset_name + '.json'))
    n_clusters = n_clusters_mapping[dataset_name]
    
    # Read dataset
    tqdm.tqdm.write(f'Processing dataset {dataset_name}...')
    tqdm.tqdm.write(f'Prior number of clusters: {n_clusters}...')
    tqdm.tqdm.write(f'Prior number of shared reads: {num_shared_read}...')

    try:
        seed_kmer_features, labels, groups, seeds = load_genomics(
            dataset,
            kmers=KMERS,
            lmer=LMER,
            maximum_seed_size=MAXIMUM_SEED_SIZE,
            num_shared_reads=num_shared_read,
            is_deserialize=is_deserialize,
            is_serialize=~is_deserialize,
            is_normalize=True,
            only_seed=ONLY_SEED,
            graph_file=os.path.join(processed_dir, dataset_name + '.json')
        )
    except:
        seed_kmer_features, labels, groups, seeds = load_genomics(
            dataset,
            kmers=KMERS,
            lmer=LMER,
            maximum_seed_size=MAXIMUM_SEED_SIZE,
            num_shared_reads=num_shared_read,
            is_deserialize=False,
            is_serialize=True,
            is_normalize=True,
            only_seed=ONLY_SEED,
            graph_file=os.path.join(processed_dir, dataset_name + '.json')
        )
    
    # continue
    # Initialize model
    init_lr = 0.1
    init = VarianceScaling(scale=1. / 3., mode='fan_in',
                               distribution='uniform')  # [-limit, limit], limit=sqrt(1./fan_in)
    pretrain_optimizer = SGD(lr=init_lr, momentum=0.9, decay=init_lr/PRETRAIN_EPOCHS)

    dec = DEC(dims=[seed_kmer_features.shape[-1], 500, 1000, 10], n_clusters=n_clusters, init=init)
    
    # Start clustering
    if AE_WEIGHTS is None:
        dec.pretrain(x=seed_kmer_features, y=labels, grps=groups, optimizer=pretrain_optimizer,
                epochs=PRETRAIN_EPOCHS, batch_size=BATCH_SIZE,
                save_dir=save_dir)
    else:
        dec.autoencoder.load_weights(AE_WEIGHTS)

    dec.model.summary()
    t0 = time.time()
    optim_lr = 0.01

    dec.compile(optimizer=SGD(optim_lr), loss='kld')
    y_pred = dec.fit(x=seed_kmer_features, y=labels, grps=groups, n_clusters=n_clusters, tol=TOL, maxiter=MAX_ITERS, batch_size=BATCH_SIZE,
                      update_interval=UPDATE_INTERVAL, save_dir=save_dir)
    
    tqdm.tqdm.write('...')
    latent = dec.encoder.predict(seed_kmer_features)
    y_pred = dec.predict(seed_kmer_features)

    tqdm.tqdm.write('Saving results...')
    store_results(groups, seed_kmer_features, latent, labels, y_pred,
                  n_clusters, dataset_name, save_dir=os.path.join(LOG_DIR, dataset_name))
    tqdm.tqdm.write(f'Finish clustering for dataset {dataset_name}.')
    tqdm.tqdm.write(f'F1-score: {genome_acc(groups, y_pred, labels, n_clusters)[2]}')
    tqdm.tqdm.write(f'Clustering time: (time.time() - t0)')


                                                                                                     
  0%|                                                                          | 0/1 [07:04<?, ?it/s]
                                                                                                     
  0%|                                                                          | 0/1 [07:04<?, ?it/s]
                                                                                                     
  0%|                                                                          | 0/1 [00:00<?, ?it/s]

Processing dataset S1...
Prior number of clusters: 2...
Prior number of shared reads: 5...
Reading fna file...
Creating document from reads...
Creating corpus...
Deserializing data...
Computing features...
Serializing data to... ../data\processed\S1.json
Normalizing...
Finish.
Epoch 1/5
152/152 [==============================] - 2s 12ms/step - loss: 1.0083
        |==>  f1-score: 0.7005 <==|
Epoch 2/5
152/152 [==============================] - 1s 8ms/step - loss: 0.9892
Epoch 3/5
152/152 [==============================] - 1s 7ms/step - loss: 0.9588
Epoch 4/5
152/152 [==============================] - 1s 8ms/step - loss: 0.9268
Epoch 5/5
152/152 [==============================] - 1s 7ms/step - loss: 0.8844
Pretraining time: 9s
Pretrained weights are saved to results\model\dec_genomics\20200530\S1/ae_weights.h5
Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           

                                                                                                     
  0%|                                                                          | 0/1 [07:59<?, ?it/s]
                                                                                                     
  0%|                                                                          | 0/1 [00:55<?, ?it/s]

Iter 1: precision = 0.98960, recall = 0.98960, f1_score = 0.98960,                            nmi = --, ari = --  ; loss= 0.04344
delta_label  0.0 < tol  1e-06
Reached tolerance threshold. Stopping training.
saving model to: results\model\dec_genomics\20200530\S1/DEC_model_final.h5
...
Saving results...
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 152 samples in 0.001s...
[t-SNE] Computed neighbors for 152 samples in 0.005s...
[t-SNE] Computed conditional probabilities for sample 152 / 152
[t-SNE] Mean sigma: 3.016403
[t-SNE] KL divergence after 250 iterations with early exaggeration: 72.761116
[t-SNE] KL divergence after 400 iterations: 0.542474
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 152 samples in 0.000s...
[t-SNE] Computed neighbors for 152 samples in 0.006s...
[t-SNE] Computed conditional probabilities for sample 152 / 152
[t-SNE] Mean sigma: 0.739312
[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.782673
[t-SNE] KL divergenc

                                                                                                     
  0%|                                                                          | 0/1 [00:58<?, ?it/s]

Visualization results are saved in:  results\log\dec_genomics\20200530\S1
Finish clustering for dataset S1.


                                                                                                     
  0%|                                                                          | 0/1 [08:03<?, ?it/s]
                                                                                                     
100%|██████████████████████████████████████████████████████████████████| 1/1 [00:59<00:00, 59.38s/it]

F1-score: 0.9896022497327923
Clustering time: (time.time() - t0)
